## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import nltk

## Import Dataset

In [2]:
df = pd.read_csv('../Netflix Recommendation System/netflix_titles.csv')

In [3]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [4]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [5]:
df['title'].head()

0     Dick Johnson Is Dead
1            Blood & Water
2                Ganglands
3    Jailbirds New Orleans
4             Kota Factory
Name: title, dtype: object

In [6]:
df['director'].head()

0    Kirsten Johnson
1                NaN
2    Julien Leclercq
3                NaN
4                NaN
Name: director, dtype: object

## Finding the Missing Values

In [7]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [8]:
df.describe(include='all')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
count,8807,8807,8807,6173,7982,7976,8797,8807.000000,8803,8804,8807,8807
unique,8807,2,8807,4528,7692,748,1767,NaN,17,220,514,8775
top,s1,Movie,Dick Johnson Is Dead,Rajiv Chilaka,David Attenborough,United States,"January 1, 2020",NaN,TV-MA,1 Season,"Dramas, International Movies","Paranormal activity at a lush, abandoned prope..."
freq,1,6131,1,19,19,2818,109,NaN,3207,1793,362,4
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.180198,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.819312,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1925.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


## Data Cleansing

In [10]:
df = df[['title', 'director', 'cast', 'listed_in', 'description']]
df['movie_info'] = df['title'] + ': ' + df['description']
df.dropna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        8807 non-null   object
 1   director     6173 non-null   object
 2   cast         7982 non-null   object
 3   listed_in    8807 non-null   object
 4   description  8807 non-null   object
 5   movie_info   8807 non-null   object
dtypes: object(6)
memory usage: 413.0+ KB


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31828\2368920525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['movie_info'] = df['title'] + ': ' + df['description']


In [11]:
df.head()

,title,director,cast,listed_in,description,movie_info
0,Dick Johnson Is Dead,Kirsten Johnson,NaN,Documentaries,"As her father nears the end of his life, filmm...",Dick Johnson Is Dead: As her father nears the ...
1,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Blood & Water: After crossing paths at a party...
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Ganglands: To protect his family from a powerf...
3,Jailbirds New Orleans,NaN,NaN,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...","Jailbirds New Orleans: Feuds, flirtations and ..."
4,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Kota Factory: In a city of coaching centers kn...


In [12]:
df.director.fillna("", inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31828\1361245944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.director.fillna("", inplace=True)


In [13]:
df.cast.fillna("",inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31828\3696203126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.cast.fillna("",inplace=True)


In [14]:
df.isnull().sum()

title          0
director       0
cast           0
listed_in      0
description    0
movie_info     0
dtype: int64

In [15]:
df.head()

,title,director,cast,listed_in,description,movie_info
0,Dick Johnson Is Dead,Kirsten Johnson,,Documentaries,"As her father nears the end of his life, filmm...",Dick Johnson Is Dead: As her father nears the ...
1,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Blood & Water: After crossing paths at a party...
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Ganglands: To protect his family from a powerf...
3,Jailbirds New Orleans,,,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...","Jailbirds New Orleans: Feuds, flirtations and ..."
4,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Kota Factory: In a city of coaching centers kn...


## Build Recommendation System

In [16]:
nltk.download('stopwords')

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc
normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['movie_info']))
len(norm_corpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


8807

#### Extract TF-IDF Features

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(8807, 20415)

#### Compute Pairwise Document Similarity

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,8797,8798,8799,8800,8801,8802,8803,8804,8805,8806
0,1.000000,0.0,0.0,0.0,0.010188,0.000000,0.000000,0.0,0.02517,0.010002,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.009286,0.000000
1,0.000000,1.0,0.0,0.0,0.000000,0.022271,0.035556,0.0,0.00000,0.000000,...,0.025141,0.0,0.040769,0.0,0.0,0.0,0.021771,0.00000,0.000000,0.000000
2,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.015696
3,0.000000,0.0,0.0,1.0,0.000000,0.000000,0.020382,0.0,0.00000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.018029,0.000000
4,0.010188,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.00000,0.011276,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.03533,0.047689,0.000000


#### Get List of Movies Titles

In [19]:
movies_list = df['title'].values
movies_list

array(['Dick Johnson Is Dead', 'Blood & Water', 'Ganglands', ...,
       'Zombieland', 'Zoom', 'Zubaan'], dtype=object)

In [20]:
movies_list.shape

(8807,)

#### Find Top Similar Movies for a Sample Movie

In [21]:
# Find movie ID
movie_id = np.where(movies_list == 'Dick Johnson Is Dead')[0][0]
movie_id

0

#### Get Similar Movies

In [22]:
movie_similarities = doc_sim_df.iloc[movie_id].values
movie_similarities

array([1.       , 0.       , 0.       , ..., 0.       , 0.0092863,
       0.       ])

#### Get Top 5 Similar Movies

In [23]:
similar_movie_id = np.argsort(-movie_similarities)[1:6]
similar_movie_id

array([5233, 7506, 4086, 4877, 1171], dtype=int64)

In [24]:
similar_movies = movies_list[similar_movie_id]
similar_movies

array(['The Death and Life of Marsha P. Johnson', 'Moon', 'Paddleton',
       'End Game', 'Seaspiracy'], dtype=object)

## Build a Movie Recommender Function to Recommend Top 5 Similar Movies For Any Movie

#### The movie title, movie title list and document similarity matrix dataframe will be given as inputs to the function

In [25]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [26]:
df.columns

Index(['title', 'director', 'cast', 'listed_in', 'description', 'movie_info'], dtype='object')

In [27]:
pop_movies = df.sort_values(by='director', ascending=False)
pop_movies.head()

,title,director,cast,listed_in,description,movie_info
982,Hayat Öpücügü,Şenol Sönmez,"Ali Sunal, Hatice Şendil, Ayda Aksel, Ahmet Sa...","Comedies, Dramas, International Movies",High-strung Metin finally stops seeing doom an...,Hayat Öpücügü: High-strung Metin finally stops...
3533,Kill Me If You Dare,Şenol Sönmez,"Murat Boz, Seda Bakan, Açelya Topaloğlu, Özgür...","Comedies, International Movies, Romantic Movies","Bogged down with money problems, a couple grow...",Kill Me If You Dare: Bogged down with money pr...
981,G.O.R.A,Ömer Faruk Sorak,"Cem Yılmaz, Rasim Öztekin, Özkan Uğur, İdil Fı...","Action & Adventure, Comedies, International Mo...",Carpet dealer and UFO photo forger Arif is abd...,G.O.R.A: Carpet dealer and UFO photo forger Ar...
6070,Aşk Tesadüfleri Sever,Ömer Faruk Sorak,"Mehmet Günsür, Belçim Bilgin, Ayda Aksel, Alta...","Dramas, International Movies, Romantic Movies",A series of coincidences brings two star-cross...,Aşk Tesadüfleri Sever: A series of coincidence...
5975,Ég man þig,Óskar Thór Axelsson,"Jóhannes Haukur Jóhannesson, Ágústa Eva Erlend...","Horror Movies, International Movies","Young urbanites renovating a rundown house, an...",Ég man þig: Young urbanites renovating a rundo...


In [28]:
df.title.values[:50]

array(['Dick Johnson Is Dead', 'Blood & Water', 'Ganglands',
       'Jailbirds New Orleans', 'Kota Factory', 'Midnight Mass',
       'My Little Pony: A New Generation', 'Sankofa',
       'The Great British Baking Show', 'The Starling',
       'Vendetta: Truth, Lies and The Mafia', 'Bangkok Breaking',
       'Je Suis Karl', 'Confessions of an Invisible Girl',
       'Crime Stories: India Detectives', 'Dear White People',
       "Europe's Most Dangerous Man: Otto Skorzeny in Spain",
       'Falsa identidad', 'Intrusion', 'Jaguar',
       'Monsters Inside: The 24 Faces of Billy Milligan',
       'Resurrection: Ertugrul', 'Avvai Shanmughi',
       'Go! Go! Cory Carson: Chrissy Takes the Wheel', 'Jeans',
       'Love on the Spectrum', 'Minsara Kanavu', 'Grown Ups',
       'Dark Skies', 'Paranoia', 'Ankahi Kahaniya', 'Chicago Party Aunt',
       'Sex Education', 'Squid Game', 'Tayo and Little Wizards',
       'The Father Who Moves Mountains', 'The Stronghold', 'Angry Birds',
       'Birth of

In [29]:
col=df.title.values[:50]

## Get Popular Movie Recommendations

In [30]:
for movie in col:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie))
    print()

Movie: Dick Johnson Is Dead
Top 5 recommended Movies: ['The Death and Life of Marsha P. Johnson' 'Moon' 'Paddleton' 'End Game'
 'Seaspiracy']

Movie: Blood & Water
Top 5 recommended Movies: ['Lion Pride' 'Message from the King' 'Walk Away from Love' 'Next Enti?'
 'Lady in the Water']

Movie: Ganglands
Top 5 recommended Movies: ['Bright' 'Narcos' 'The River Thief' 'Miss Dynamite'
 "The Eagle of El-Se'eed"]

Movie: Jailbirds New Orleans
Top 5 recommended Movies: ['Project Power' 'The Originals' 'The Runner' 'Cut Throat City'
 'The Haunted Hathaways']

Movie: Kota Factory
Top 5 recommended Movies: ['The Roommate' 'Sexify' 'Dude' 'Criminal Minds' 'Train of the Dead']

Movie: Midnight Mass
Top 5 recommended Movies: ['There Will Be Blood' 'Holy Goalie' 'Until Midnight'
 'La Rosa de Guadalupe' 'Midnight Special']

Movie: My Little Pony: A New Generation
Top 5 recommended Movies: ['My Little Pony Equestria Girls: Legend of Everfree'
 'My Little Pony Friendship Is Magic: Best Gift Ever'
 'My Li